In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.pyplot import subplots, figure, show
from matplotlib import pyplot, pylab
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
#from sklearn.grid_search import GridSearchCV

from sklearn import metrics
from sklearn.svm import SVC

In [2]:
df=pd.read_csv("data/MCI_2014_to_2019.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206435 entries, 0 to 206434
Data columns (total 29 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   X                    206435 non-null  float64
 1   Y                    206435 non-null  float64
 2   Index_               206435 non-null  int64  
 3   event_unique_id      206435 non-null  object 
 4   occurrencedate       206435 non-null  object 
 5   reporteddate         206435 non-null  object 
 6   premisetype          206435 non-null  object 
 7   ucr_code             206435 non-null  int64  
 8   ucr_ext              206435 non-null  int64  
 9   offence              206435 non-null  object 
 10  reportedyear         206435 non-null  int64  
 11  reportedmonth        206435 non-null  object 
 12  reportedday          206435 non-null  int64  
 13  reporteddayofyear    206435 non-null  int64  
 14  reporteddayofweek    206435 non-null  object 
 15  reportedhour     

In [4]:
df.head()

,X,Y,Index_,event_unique_id,occurrencedate,reporteddate,premisetype,ucr_code,ucr_ext,offence,...,occurrencedayofyear,occurrencedayofweek,occurrencehour,MCI,Division,Hood_ID,Neighbourhood,Long,Lat,ObjectId
0,-79.405228,43.656982,7801,GO-20152165447,2015-12-18T03:58:00.000Z,2015-12-18T03:59:00.000Z,Commercial,1430,100,Assault,...,352.0,Friday,3,Assault,D14,79,University (79),-79.405228,43.656982,7001
1,-79.307907,43.778732,7802,GO-20151417245,2015-08-15T21:45:00.000Z,2015-08-17T22:11:00.000Z,Commercial,1430,100,Assault,...,227.0,Saturday,21,Assault,D42,118,Tam O'Shanter-Sullivan (118),-79.307907,43.778732,7002
2,-79.225029,43.765942,7803,GO-20151421107,2015-08-16T16:00:00.000Z,2015-08-18T14:40:00.000Z,Apartment,2120,200,B&E,...,228.0,Sunday,16,Break and Enter,D43,137,Woburn (137),-79.225029,43.765942,7003
3,-79.140823,43.778648,7804,GO-20152167714,2015-11-26T13:00:00.000Z,2015-12-18T13:38:00.000Z,Other,2120,200,B&E,...,330.0,Thursday,13,Break and Enter,D43,133,Centennial Scarborough (133),-79.140823,43.778648,7004
4,-79.288361,43.691235,7805,GO-20152169954,2015-12-18T19:50:00.000Z,2015-12-18T19:55:00.000Z,Commercial,1430,100,Assault,...,352.0,Friday,19,Assault,D55,61,Taylor-Massey (61),-79.288361,43.691235,7005


In [5]:
#based on the previos corelation 

In [6]:
df = pd.read_csv('data/MCI_2014_to_2019.csv',sep=',') 
df = df.drop(df[df['occurrenceyear'] < 2014].index)
data_org = df.iloc[:,[22,6,17,20,21,25]]
data_org = data_org.dropna()

In [7]:
data_org

,MCI,premisetype,occurrencemonth,occurrencedayofweek,occurrencehour,Neighbourhood
0,Assault,Commercial,December,Friday,3,University (79)
1,Assault,Commercial,August,Saturday,21,Tam O'Shanter-Sullivan (118)
2,Break and Enter,Apartment,August,Sunday,16,Woburn (137)
3,Break and Enter,Other,November,Thursday,13,Centennial Scarborough (133)
4,Assault,Commercial,December,Friday,19,Taylor-Massey (61)
...,...,...,...,...,...,...
206430,Auto Theft,Outside,February,Thursday,23,Willowridge-Martingrove-Richview (7)
206431,Auto Theft,Commercial,February,Thursday,0,South Riverdale (70)
206432,Auto Theft,House,January,Wednesday,12,West Humber-Clairville (1)
206433,Auto Theft,Commercial,February,Thursday,0,South Riverdale (70)


In [8]:
data = data_org.copy()
# Get one hot encoding of columns premisetype
one_hot = pd.get_dummies(data['premisetype'])
# Join the encoded df
data = data.join(one_hot)

# Get one hot encoding of columns occurrencemonth
one_hot = pd.get_dummies(data['occurrencemonth'])
# Join the encoded df
data = data.join(one_hot)

# Get one hot encoding of columns occurrencedayofweek
one_hot = pd.get_dummies(data['occurrencedayofweek'])
# Join the encoded df
data = data.join(one_hot)

# Get one hot encoding of columns occurrencehour
one_hot = pd.get_dummies(data['occurrencehour'])
# Join the encoded df
data = data.join(one_hot)

# Get one hot encoding of columns Neighbourhood
one_hot = pd.get_dummies(data['Neighbourhood'])
# Join the encoded df
data = data.join(one_hot)

# Set column Neighbourhood as target
data = data.drop('premisetype',axis = 1)
data = data.drop('occurrencemonth',axis = 1)
data = data.drop('Neighbourhood',axis = 1)
data = data.drop('occurrencedayofweek',axis = 1)
data = data.drop('occurrencehour',axis = 1)

In [9]:
data

,MCI,Apartment,Commercial,House,Other,Outside,April,August,December,February,...,Willowdale West (37),Willowridge-Martingrove-Richview (7),Woburn (137),Woodbine Corridor (64),Woodbine-Lumsden (60),Wychwood (94),Yonge-Eglinton (100),Yonge-St.Clair (97),York University Heights (27),Yorkdale-Glen Park (31)
0,Assault,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,Assault,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Break and Enter,1,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
3,Break and Enter,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Assault,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206430,Auto Theft,0,0,0,0,1,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0
206431,Auto Theft,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
206432,Auto Theft,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
206433,Auto Theft,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [10]:
data2=data.copy()
data2=data2.drop('MCI',axis=1)


In [11]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(data2)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [12]:
# Target selection
y = data['MCI']
data = data.drop('MCI',axis = 1)
# Features selection
X = data[data2.columns[:]]  

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4) # 70% training and 30% test

In [13]:
#Logistic Regression

In [14]:
from sklearn.linear_model import LogisticRegression
import category_encoders as ce
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline

lr= make_pipeline(
    ce.OrdinalEncoder(), 
    SimpleImputer(strategy='mean'), 
    LogisticRegression(n_jobs=-1,solver='lbfgs', random_state=42, penalty='l2')
)

lr.fit(X_train, y_train);

In [15]:
from sklearn.linear_model import LogisticRegression
lgr= LogisticRegression()

lgr.fit(X_train,y_train) #Training

predic_y1= lgr.predict(X_test) #Predicting y values of x test based on training
predic_y1

# Checking accuracy
from sklearn.metrics import accuracy_score
logistic_accuracy=accuracy_score(y_test,predic_y1)*100
print(f'\n Accuracy of Logistic Regression {logistic_accuracy}%.')

C:\Users\jilsj\Anaconda3\envs\project\lib\site-packages\sklearn\linear_model\_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



 Accuracy of Logistic Regression 58.772175480037504%.


In [17]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
kn=KNeighborsClassifier(n_neighbors=5,n_jobs=-1)# using full cores
kn.fit(X_train,y_train)
#predic_y2=kn.predict(X_test)#Output Prediction
#predic_y2
#acc_train = kn.score(X_train, y_train)
#acc_test = kn.score(X_test, y_test)
y_pred = kn.predict(X_test)
y_training = kn.predict(X_train)
#print("Training set accuracy: {:.2f}".format(acc_train))  
#print("Test set accuracy: {:.2f}".format(acc_test))
print("Testing Accuracy:", round(metrics.accuracy_score(y_test, y_pred) * 100, 2), "%")
print("Training Accuracy:", round(metrics.accuracy_score(y_train, y_training)* 100, 2), "%\n")
print( 'Classification report: \n', metrics.classification_report(y_test, y_pred), "\n")

Testing Accuracy: 56.91 %
Training Accuracy: 68.72 %

Classification report: 
                  precision    recall  f1-score   support

        Assault       0.61      0.83      0.70     44053
     Auto Theft       0.39      0.26      0.31      9440
Break and Enter       0.52      0.38      0.44     17435
        Robbery       0.37      0.14      0.21      8504
     Theft Over       0.06      0.01      0.01      2697

       accuracy                           0.57     82129
      macro avg       0.39      0.32      0.33     82129
   weighted avg       0.52      0.57      0.53     82129
 



In [18]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
kn=KNeighborsClassifier(n_neighbors=1,n_jobs=-1)# using full cores
kn.fit(X_train,y_train)
#predic_y2=kn.predict(X_test)#Output Prediction
#predic_y2
#acc_train = kn.score(X_train, y_train)
#acc_test = kn.score(X_test, y_test)
y_pred = kn.predict(X_test)
y_training = kn.predict(X_train)
#print("Training set accuracy: {:.2f}".format(acc_train))  
#print("Test set accuracy: {:.2f}".format(acc_test))
print("Testing Accuracy:", round(metrics.accuracy_score(y_test, y_pred) * 100, 2), "%")
print("Training Accuracy:", round(metrics.accuracy_score(y_train, y_training)* 100, 2), "%\n")
print( 'Classification report: \n', metrics.classification_report(y_test, y_pred), "\n")

Testing Accuracy: 52.65 %
Training Accuracy: 94.38 %

Classification report: 
                  precision    recall  f1-score   support

        Assault       0.65      0.67      0.66     44053
     Auto Theft       0.35      0.33      0.34      9440
Break and Enter       0.46      0.41      0.44     17435
        Robbery       0.33      0.35      0.34      8504
     Theft Over       0.06      0.06      0.06      2697

       accuracy                           0.53     82129
      macro avg       0.37      0.37      0.37     82129
   weighted avg       0.52      0.53      0.52     82129
 



In [19]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
kn=KNeighborsClassifier(n_neighbors=3,n_jobs=-1)# using full cores
kn.fit(X_train,y_train)
#predic_y2=kn.predict(X_test)#Output Prediction
#predic_y2
#acc_train = kn.score(X_train, y_train)
#acc_test = kn.score(X_test, y_test)
y_pred = kn.predict(X_test)
y_training = kn.predict(X_train)
#print("Training set accuracy: {:.2f}".format(acc_train))  
#print("Test set accuracy: {:.2f}".format(acc_test))
print("Testing Accuracy:", round(metrics.accuracy_score(y_test, y_pred) * 100, 2), "%")
print("Training Accuracy:", round(metrics.accuracy_score(y_train, y_training)* 100, 2), "%\n")
print( 'Classification report: \n', metrics.classification_report(y_test, y_pred), "\n")

Testing Accuracy: 55.94 %
Training Accuracy: 73.71 %

Classification report: 
                  precision    recall  f1-score   support

        Assault       0.61      0.81      0.69     44053
     Auto Theft       0.38      0.25      0.30      9440
Break and Enter       0.50      0.37      0.42     17435
        Robbery       0.37      0.20      0.26      8504
     Theft Over       0.08      0.01      0.02      2697

       accuracy                           0.56     82129
      macro avg       0.39      0.33      0.34     82129
   weighted avg       0.52      0.56      0.52     82129
 



In [20]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pca.fit(X_train)
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)
print("original shape of train :   ", X.shape)
print("transformed shape of train :", X_train_pca.shape)
print("transformed shape of test :", X_test_pca.shape)

original shape of train :    (205321, 188)
transformed shape of train : (123192, 2)
transformed shape of test : (82129, 2)


In [46]:
from sklearn.linear_model import LogisticRegression
lgr= LogisticRegression()

lgr.fit(X_train,y_train) #Training

predic_y1= lgr.predict(X_test) #Predicting y values of x test based on training
predic_y1

# Checking accuracy
from sklearn.metrics import accuracy_score
logistic_accuracy=accuracy_score(y_test,predic_y1)*100
print(f'\n Accuracy of Logistic Regression {logistic_accuracy}%.')


 Accuracy of Logistic Regression 58.71389840414306%.


C:\Users\jilsj\Anaconda3\envs\project\lib\site-packages\sklearn\linear_model\_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [65]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
kn=KNeighborsClassifier(n_neighbors=3,n_jobs=-1)# using full cores
kn.fit(X_train,y_train)
#predic_y2=kn.predict(X_test)#Output Prediction
#predic_y2
#acc_train = kn.score(X_train, y_train)
#acc_test = kn.score(X_test, y_test)
y_pred = kn.predict(X_test)
y_training = kn.predict(X_train)
#print("Training set accuracy: {:.2f}".format(acc_train))  
#print("Test set accuracy: {:.2f}".format(acc_test))
print("Testing Accuracy:", round(metrics.accuracy_score(y_test, y_pred) * 100, 2), "%")
print("Training Accuracy:", round(metrics.accuracy_score(y_train, y_training)* 100, 2), "%\n")
print( 'Classification report: \n', metrics.classification_report(y_test, y_pred), "\n")

Accuracy of KNN algorithm is 53.356494634478956%.


Training set accuracy: 0.94
Test set accuracy: 0.53


In [21]:
new_mci=['Assault','Auto Theft','Break and Enter']
data_org1=data_org[data_org.MCI.isin(new_mci)]


In [22]:
data = data_org1.copy()
# Get one hot encoding of columns premisetype
one_hot = pd.get_dummies(data['premisetype'])
# Join the encoded df
data = data.join(one_hot)

# Get one hot encoding of columns occurrencemonth
one_hot = pd.get_dummies(data['occurrencemonth'])
# Join the encoded df
data = data.join(one_hot)

# Get one hot encoding of columns occurrencedayofweek
one_hot = pd.get_dummies(data['occurrencedayofweek'])
# Join the encoded df
data = data.join(one_hot)

# Get one hot encoding of columns occurrencehour
one_hot = pd.get_dummies(data['occurrencehour'])
# Join the encoded df
data = data.join(one_hot)

# Get one hot encoding of columns Neighbourhood
one_hot = pd.get_dummies(data['Neighbourhood'])
# Join the encoded df
data = data.join(one_hot)

# Set column Neighbourhood as target
data = data.drop('premisetype',axis = 1)
data = data.drop('occurrencemonth',axis = 1)
data = data.drop('Neighbourhood',axis = 1)
data = data.drop('occurrencedayofweek',axis = 1)
data = data.drop('occurrencehour',axis = 1)

In [23]:
#Target selection
y = data['MCI']
data = data.drop('MCI',axis = 1)
# Features selection
X = data[data.columns[:]]  
# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) # 70% training and 30% test

In [21]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
kn=KNeighborsClassifier(n_neighbors=1,n_jobs=-1)# using full cores
kn.fit(X_train,y_train)
acc_train = kn.score(X_train, y_train)
acc_test = kn.score(X_test, y_test)

print("Training set accuracy: {:.2f}".format(acc_train))  
print("Test set accuracy: {:.2f}".format(acc_test))


Training set accuracy: 0.96
Test set accuracy: 0.63


In [23]:
kn=KNeighborsClassifier(n_neighbors=3,n_jobs=-1)# using full cores
kn.fit(X_train,y_train)
acc_train = kn.score(X_train, y_train)
acc_test = kn.score(X_test, y_test)

print("Training set accuracy: {:.2f}".format(acc_train))  
print("Test set accuracy: {:.2f}".format(acc_test))

Training set accuracy: 0.81
Test set accuracy: 0.65


In [24]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
kn=KNeighborsClassifier(n_neighbors=3,n_jobs=-1)# using full cores
kn.fit(X_train,y_train)
#predic_y2=kn.predict(X_test)#Output Prediction
#predic_y2
#acc_train = kn.score(X_train, y_train)
#acc_test = kn.score(X_test, y_test)
y_pred = kn.predict(X_test)
y_training = kn.predict(X_train)
#print("Training set accuracy: {:.2f}".format(acc_train))  
#print("Test set accuracy: {:.2f}".format(acc_test))
print("Testing Accuracy:", round(metrics.accuracy_score(y_test, y_pred) * 100, 2), "%")
print("Training Accuracy:", round(metrics.accuracy_score(y_train, y_training)* 100, 2), "%\n")
print( 'Classification report: \n', metrics.classification_report(y_test, y_pred), "\n")

Testing Accuracy: 65.5 %
Training Accuracy: 81.18 %

Classification report: 
                  precision    recall  f1-score   support

        Assault       0.71      0.82      0.76     33215
     Auto Theft       0.47      0.32      0.38      6968
Break and Enter       0.53      0.42      0.47     12963

       accuracy                           0.66     53146
      macro avg       0.57      0.52      0.54     53146
   weighted avg       0.64      0.66      0.64     53146
 



In [25]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
kn=KNeighborsClassifier(n_neighbors=1,n_jobs=-1)# using full cores
kn.fit(X_train,y_train)
#predic_y2=kn.predict(X_test)#Output Prediction
#predic_y2
#acc_train = kn.score(X_train, y_train)
#acc_test = kn.score(X_test, y_test)
y_pred = kn.predict(X_test)
y_training = kn.predict(X_train)
#print("Training set accuracy: {:.2f}".format(acc_train))  
#print("Test set accuracy: {:.2f}".format(acc_test))
print("Testing Accuracy:", round(metrics.accuracy_score(y_test, y_pred) * 100, 2), "%")
print("Training Accuracy:", round(metrics.accuracy_score(y_train, y_training)* 100, 2), "%\n")
print( 'Classification report: \n', metrics.classification_report(y_test, y_pred), "\n")

Testing Accuracy: 63.59 %
Training Accuracy: 96.23 %

Classification report: 
                  precision    recall  f1-score   support

        Assault       0.72      0.76      0.74     33215
     Auto Theft       0.41      0.39      0.40      6968
Break and Enter       0.50      0.45      0.47     12963

       accuracy                           0.64     53146
      macro avg       0.54      0.53      0.54     53146
   weighted avg       0.63      0.64      0.63     53146
 

